#### From 01.Twitter Sentiment


##### https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/

In [1]:
import re
import tweepy
import config
from tweepy import OAuthHandler
from textblob import TextBlob

In [2]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = config.consumer_key
        consumer_secret = config.consumer_secret
        access_token = config.access_token
        access_token_secret = config.access_token_secret
  
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
  
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
  
    def get_tweets(self, query, count = 30):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
  
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search_tweets(q = query, lang = "en" , count = count)
            
            
            #print(fetched_tweets)
            
            return fetched_tweets
  
        except tweepy.Error as e:
            
            print("Error : " + str(e))

In [3]:
def clean_tweet(tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
  


  

In [4]:
def get_tweet_sentiment(tweet):
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet))
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

In [5]:
 # Extract hashtags
def read_hashtags(tag_list):
    hashtags = []
    for tag in tag_list:
        hashtags.append(tag['text'])
    return hashtags

In [6]:
import psycopg2
import config


# Insert Tweet data into database
def dbConnect(user_id, user_id_str, user_name, screen_name, location, URL, description,verified, created_at, tweet_id, tweet_id_str, tweet, retweet_count, hashtags):
    
    # Connection to database server
    #conn = psycopg2.connect(host="localhost",database="TwitterDB",port=5432,user='postgres',password=config.db_password)
    conn = psycopg2.connect(host=config.PGHOST,database="postgres",port=5432,user='postgres',password=config.db_password)
    cur = conn.cursor()

    # insert user information
    command = '''INSERT INTO TwitterUser (user_id, user_id_str, user_name, screen_name, location , URL, description,verified) VALUES (%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT
                 (User_Id) DO NOTHING;'''
    cur.execute(command,(user_id, user_id_str, user_name, screen_name, location , URL, description,verified))

    # insert tweet information
    command = '''INSERT INTO TwitterTweet (created_at, tweet_id, id_str, user_id, tweet, retweet_count) VALUES (%s,%s,%s,%s,%s,%s);'''
    cur.execute(command,(created_at, tweet_id, tweet_id_str, user_id, text, retweet_count))
    
    # insert entity information
    for i in range(len(hashtags)):
        hashtag = hashtags[i]
        command = '''INSERT INTO TwitterEntity (tweet_id, hashtag) VALUES (%s,%s);'''
        cur.execute(command,(tweet_id, hashtag))
    
    # Commit changes
    conn.commit()
    
    # Disconnect
    cur.close()
    conn.close()
    

In [7]:
api = TwitterClient()
#tweets = api.get_tweets(query = ['#Covid19','#Covid-19','#coronavirus'], count = 30)
#tweets = api.get_tweets(query = ['#COVID19','#Covid19','#COVID19','#Covid-19','#COVID-19','Covid','COVID-19','Pfizer/BioNTech','Sinopharm', 'moderna','pfizer','Sinovac','Oxford/AstraZeneca','Covaxin','Sputnik V'], count = 500)
tweets = api.get_tweets(query = ['#Covid19','#COVID19','#Covid-19','#COVID-19'], count = 500) #this is the one that works
#tweets = api.get_tweets(query = ['Pfizer/BioNTech'], count = 500)


for tweet in tweets:
    
    #print(dir(tweet))
    #print(tweet)
    
    
    user_id = tweet.user.id
    user_id_str = tweet.user.id_str
    user_name = tweet.user.name
    screen_name = tweet.user.screen_name
    location = tweet.user.location
    URL = tweet.user.url
    description = tweet.user.description
    verified = tweet.user.verified
    tweet_url = None

    
    followers = tweet.user.followers_count
    following = tweet.user.following
    
    user_creation_date = tweet.user.created_at
    
    
    
    
    created_at = tweet.created_at
    tweet_id = tweet.id
    tweet_id_str = tweet.id_str
    #tweet_url = tweet.tweet_url
    text = tweet.text
    source = tweet.source
    truncated = tweet.truncated
    in_reply_to_status_id = tweet.in_reply_to_status_id
    
    in_reply_to_user_id_str = tweet.in_reply_to_user_id_str 
    in_reply_to_user_id = tweet.in_reply_to_user_id 

    in_reply_to_screen_name = tweet.in_reply_to_screen_name 
    user = tweet.user.id
    likes = tweet.favorite_count
    
    if tweet.coordinates is None:
        coord_long = None
        coord_lat = None
    else:
        coord_long = tweet.coordinates['coordinates'][0]
        coord_lat = tweet.coordinates['coordinates'][1]
    
    

    
    
    #place
    
    

    is_quote_status = tweet.is_quote_status
    
    retweet = tweet. retweet
    retweets = tweet.retweets
    retweet_count = tweet.retweet_count

    
    favorite_count = tweet.favorite_count

    
    favorited = tweet.favorited
    retweeted = tweet.retweeted
    #possibly_sensitive = tweet.possibly_sensitive
    #filter_level = tweet.filter_level
    lang = tweet.lang
    
    
    if tweet.truncated == True:
        #tweet = status.extended_tweet['full_text']
        hashtags = tweet.entities['hashtags']
        #hashtags = tweet.extended_tweet['entities']['hashtags']
    else:
        #tweet = status.text
        hashtags = tweet.entities['hashtags']

    # Read hashtags
    hashtags = read_hashtags(hashtags)        
    
    
    
    
    
    sentiment = get_tweet_sentiment(clean_tweet(tweet.text))
   
    
    #hashtags = tweet.entities['hashtags'][0]
    
    #if not hasattr(status, "retweeted_status") and lang=="en":
    if lang == "en":   
        # Connect to database
        dbConnect(user_id, user_id_str, user_name, screen_name, location, URL, description, verified, created_at, tweet_id, tweet_id_str, text, retweet_count, hashtags)
        
        

    
    
    
#print(tweets)    
    #print(tweet.entities['hashtags'])

KeyboardInterrupt: 